** убрать все внешние картинки **

** для всего кода прямо над клеткой ставить формулы**

# Линейная регрессия

## Почему мы изучаем ее, а не глубокие нейронные сети?

В данном уроке мы изучим основные концепции и идеи, которые широко применяются в машинном обучении. Линейная регрессия достаточно простая для понимания, и она до сих пор применяется во многих задачах.

** Добавить больше зачем они нужны и где применяеются сейчас. Хотя бы эксель и ClickHouse.**

** Рассказать что есть из коробки. Но важно понимать инструмент. **

** Расказать про sklearn **

## Постановка задачи

Представим, что мы хотим определить цену в одном доме квартиры на основании площади квартиры. 
У нас есть база данных полученная от риэлтерской компании. 
В данном случае нам нужно найти зависимость между площадью квартиры и ценой квартиры. 

То есть, нужно найти функцию $f(X) = y$, где $X$ это площадь квартиры, а $y$ это цена на квартиру. Это и есть задача регрессии.


Пример: 
*Нам говорят, у квартиры площадь 45 квадратных метров. Мы берем это значение, помещаем в нашу функцию и получаем значение.* 

$f(45) = 10 000 000$

*И мы говорим в ответ: она будет стоит 10 млн рублей.*

** Больше про функции **

**Другой пример регрессии.** У нас есть интернет сайт и мы хотим предсказать сколько посититлей у нас будет в в определенное время, при этом у нас есть статистика посещений сайта от времени. Наша задача - используя имеющиеся данные о загруженности в прошлом, предсказать количество поситителей в интересующиее нас время. То есть по имеющимся данным построить зависимость интересующией нас переменной от каких то других параметров.

В общем случае задача регрессии - это по входным данным найти число в каком-то *непрерывном* диапазоне. Например: $(-1, 1)$ или же $R$.

Рассмотрим подробнее пример с ценами на квартиру.
Давайте загрузим. После этого в вектре $X$ у нас будет площадь квартиры в $м^2$, а в вектре $y$ цена на квартиру. 

** Добавить bias к данным и уменьшить таргет на миллион, что бы уменшить числа **

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from regression_helper import * # Подгружаем функции для визуализации
import numpy as np              # Подгруджаем библиотеку NumPy

X, y = get_data()               # Загружаем данные в X и y

** Чуть больше показать данные **

Отобразим эти точки на графике

In [ ]:
plot_data(X, y)      # Строим диаграму с точками

In [ ]:
visualize_X(X)    

In [ ]:
visualize_y(y)

Давайте посмотрим на данные

In [ ]:
print(X)

$X_i$ в тексте - это ***X[i]*** в коде.

In [ ]:
print(X[0])

In [ ]:
print(y)

Видим, что чем больше площадь квартиры, тем выше ее цена. Можно сделать вывод, что между этими данными есть зависимость.

Допустим мы предполагаем, что данная зависимость может описаться линейной функцией вида $y = kX$, где $X$ это площадь квартиры, а $y$ ее цена, $k$ - коэффициент который мы попытаемся подобрать.

Мы предполагаем, что данную зависимость можно описать такой функцией, это наша гипотеза. 
Давайте нанесем на график несколько линейных функций с разным коэффициентом что бы убедится, что такой вид функции подходит.

In [ ]:
choose_slope(X, y)

Реализуем эту функцию.

Перед каждой функций будем писать что конкретно мы реализуем.

Самая простая реализация в самом простом виде выглядит так:

Для каждого элемента $X_i$ массива $X$ реализовать фукнуию $f(X_i) = kX_i$.

* На входе массив $X$

* На выходе массив со значениями $f(x)$

In [ ]:
def dummy_realization(X, k):
    result = []            # Массив 
    for x in X:
        result.append(k*x)
    return np.array(result)

print(dummy_realization(X, k=100))

Это не самая лучшая реализация. 

Можно воспользоваться возможностью питона и сделать код покороче.

Для каждого элемента $X_i$ массива $X$ реализовать фукнуию $f(X_i) = kX_i$.

* На входе массив $X$

* На выходе массив со значениями $f(x)$

In [ ]:
def less_dummy_realization(X, k):
    return np.array([k*x for x in X])

print(less_dummy_realization(X, k=100))

Для каждого элемента $X_i$ массива $X$ реализовать фукнуию $f(X_i) = kX_i$.

* На входе массив $X$

* На выходе массив со значениями $f(x)$

In [ ]:
def f(X, k):
    return k*X

print(f(X=X, k=100))

При этом, эта функция также работает и для одного элемента.

In [ ]:
print(f(X=X[0], k=100))

### Разница во времени выполнения на 100000 элементов в ndarray

Функция                | Среднее время (микросекунд) 
-----------------------|-----------------------------
dummy_realization      | 31400                       
less_dummy_realization | 13100                       
f                      | 70                          

## Функция ошибки

У нас есть гипотезы. Но как численно определить какая из них лучшая?

Для это введем функцию ошибку, также известную как функцию потерь. Функция ошибки - численное значение того, какая разница между функцией которой мы получили и данными. Обозначается функция потерь как $Loss$, $L$ или $J$. 

Лучше всего не использовать значение $L$. 

$L$ - часто используется как обозначение для функции правдопадобия, которая часто используется в машинном обучении.

Давайте визуализируем ошибки для наших гипотез.

In [ ]:
plot_data_and_error(X, y)

** Каждая формула на отдельной строке **

Пусть у нас есть функция (наша модель):  

$\widetilde{y} = f(X) = kX$

То есть, $\widetilde{y}$ является предсказанными нами значениями для $X$. А настоящие значения будут равны $y$. Тогда ошибку нашего предсказания на $i$-ом примере $\widetilde{y}_i$ можно посчитать, как: 

$error = \widetilde{y}_i - y_i$

Если нам не важно в большую или в меньшую сторону мы ошибаемся, можем избавиться от знака, взяв либо модуль либо квадрат ошибки. Для начала давайте возьмем квадрат. О модуле мы поговорим во втором уроке регрессии. Тогда ошибка $j(k)$ на i-ом примере будет равна:

$j(k) = (\widetilde{y}_i - y_i)^2$

$j(k) = (kX_i - y_i)^2$

In [ ]:
k = 10
for i in range(X.shape[0]): 
    diff = f(X[i], k) - y[i]
    print(f"Разница на примере {i} равна {diff:.4}")

In [ ]:
k = 10
for i in range(X.shape[0]): 
    diff_quad = (f(X[i], k) - y[i])**2
    print(f"Квадрат разницы на примере {i} равен {diff_quad:.4}")

In [ ]:
k = 10
print(f(X, k) - y)

In [ ]:
k = 10
print((f(X, k) - y)**2)

Но это на одном примере. А на всех?

**Разбить формулы **

Тогда мы можем посчитать среднюю ошибку $J(k)$ на всех примерах:

$J(k) = \dfrac{1}{N} \sum_{i=0}^{N}{(\widetilde{y}_i - y_i)^2}$ 

$J(k) = \dfrac{1}{N} \sum_{i=0}^{N}{(f(X_i) - y_i)^2}$

$J(k) = \dfrac{1}{N} \sum_{i=0}^{N}{(kX_i - y_i)^2} $

Где $N$ - это колличество примеров, $X_i$ - это площадь i-oй квартиры, $y_i$ - цена для i-oй квартиры, $\widetilde{y}_i$ - предсказанная цена для i-oй квартиры.

***Немного про нотацию*** 

Функция потерь формально зависит от входных данных, реальных выходных данных, вида функции и параметров этой функции. 

То есть, $J(X, y, f, k)$. Но обычно, для краткости, мы предполагаем, что мы используем текущие $X$, $y$ и $f$ и пишем, что функция потерь зависит только от параметров функции.

Для входных данных массива $X$ и реального выходного значения $y$ реализовать функцию ошибки. 

На входе:

* Коэффициент $k$ функции $f$

* Массив входных значений $X$

* Массив реальных (верных) выходных значений $y$

На выходе:

* На выходе значение фуникции ошибки $J(k)$

$J(k) = \dfrac{1}{N} \sum_{i=0}^{N}{(kX_i - y_i)^2} $

In [ ]:
def loss_function(k, X, y):
    
    N = X.shape[0]                    # получаем размер вектора столбца
    # или N = len(X)
   
    J_for_sample = (k*X - y)**2
    J = np.sum(J_for_sample) / N      
    
    # или J = np.mean((k*X - y)**2)
    return J

k = 10
print(loss_function(k, X, y))

In [ ]:
plot_data_and_J(X, y)

Давайте теперь визуализируем всю функцию ошибки для всех функция значений $k$.

**Объяснить что это не аналитическая форма. Расказать что это  создано фором и можно сделать только для данного случая **

In [ ]:
plot_all_J(X, y)

## Производная

Наша задачи - это минимизации функции ошибки. То есть, нужно найти такое $k$, для которого фунция $J(k)$ имеет минимальное значение.

Для того что бы это сделать придется немного потерпеть и вспомнить математику. 

Давайте повторим что такое *производная функции*.

Производная функции $f(x)$ записывается как $f'(x)$ или же как $\dfrac{d f(x)}{dx}$

Сама производная это то, насколько значение функции меняется в зависимости от изменения входного значения. 

В данном случае для какой-то точки $x_0$ производную можно рассматривать 

$f'(x_0) = \dfrac{d f(x_0)}{dx} $

$f'(x_0) = \dfrac{f(x_0+\Delta x) - f(x_0)}{\Delta x}$

$\Delta x \rightarrow 0$

Посмотрим на функцию $f(x) = x^2 + 1.5$

В какой-то точке $x_0$ производная будет равна:

$f'(x_0) = \dfrac{f(x_0+\Delta x) - f(x_0)}{\Delta x}$


$f(x_0+\Delta x) = (x_0 + \Delta x)^2 + 1.5 = x_0^2 + 2x_0\Delta x + \Delta x^2 + 1.5$


$ \dfrac{f(x_0+\Delta x) - f(x_0)}{\Delta x} = \dfrac{x_0^2 + 2x_0\Delta x + \Delta x^2 + 1.5 - (x_0^2 + 1.5) }{\Delta x} = \dfrac{2x_0\Delta x + \Delta x^2}{\Delta x}$


$\dfrac{2x_0\Delta x + \Delta x^2}{\Delta x} = 2x_0 + \Delta x$

И если мы будем счиать $\Delta x$ очень маленьким числом, тогда 

$f'(x_0) = 2x_0$

In [ ]:
x0 = 1.4
derivation(x0=x0)

In [ ]:
x0 = -1.3
derivation(x0=x0)

Можно считать, что производная в точке показывает "скорость" изменения функции. Она положительна если функция растет и отрицательно, когда убывает.

Еще ее можно интерпретировать как угол наклона $\alpha$ касательной.

Обычно пишут что $f'(x) = tg(\alpha)$.

Рассмотрим повевдение производной в точках перегиба.

Точки перегиба - это, например, когда функция принимает максимальные и минимальные значения.

In [ ]:
derivation(x0=0)

Точках перегиба, производная равна 0. 

Расмотрим функцию и ее производную.

In [ ]:
plot_func_and_der(False)

При взятии производной сложной функции использоваться chain rule:

**Chain рул нужно расписать до интуции **

$u = \phi(x)$

$y= f(u)$

$\frac{dy}{dx} = \frac{df(u)}{du} \cdot \frac{du(x)}{dx}$

Производная суммы/разности равна сумме/разности производных:

** Расписать все правила и добавить примеры произоводных для этих правил  **

$(f_1(x) + f_2(x))' = f_1'(x) + f_2'(x)$

### Пример 1 ###

$y = sin(3x - 5)$

$u = 3x - 5$

$y = sin(u)$

$\frac{dy}{dx} = \frac{d sin(u)}{du} \cdot \frac{d(3x - 5)}{dx}$ 


$\frac{d sin(u)}{du} = cos(u)$

$\frac{d(3x - 5)}{dx} = 3$


$\frac{dy}{dx} = 3cos(3x - 5)$

### Пример 2 ###

$y = (10x^2 + x)^3$

$u = 10x^2 + x$

$y = u^3$

$\frac{dy}{dx} = \frac{d  u^3}{du} \cdot \frac{d(10x^2 + x)}{dx}$ 


$\frac{d  u^3}{du} = 3u^2$

$\frac{d(10x^2 + x)}{dx} = 20x + 1$

$\frac{dy}{dx} = 3u^2 \cdot(20x + 1) = 3(10x^2 + x)^2 \cdot(20x + 1) $

Таким образом, в нашем случае:

$ J'(k) = \dfrac{dJ(k)}{dk} = \dfrac{\dfrac{1}{N}\sum_{i=1}^{N}{(y_i - \widetilde{y}_i)^2}}{dk} $

$\dfrac{dJ(k)}{dk} = 2 \cdot \dfrac{1}{N}\sum_{i=1}^{N} (kX_i - y_i)\dfrac{d(kX_i - y_i)}{dk} $

$\dfrac{dJ(k)}{dk} =  \dfrac{2}{N} \sum_{i=1}^{N} (kX_i - y_i)X_i$ 
          
И для нахождения минимума нужно приравнять производную к нулю и решаем уравнение относительно k. 

$ \dfrac{dJ(k)}{dk} = 0$

Решив уравнение, мы получим значение для $k=$

Я думаю, что вы знаете как можно найти значение минимума для данной функции. Нужно взять производную функции ошибки и приравнять ее к нулю. $J'(k) = \frac{dJ(k)}{dk} = 0$.

In [ ]:
plot_data_and_hyp(X, y, 1.0)

Но в реальных приложениях не всегда возможно решить это уравнение (далее я расскажу поподробнее о таких ситуациях). 

Поэтому, познакомимся с таким алгоритмом как градиентный спуск. 

Данный алгоритм является одним самых распространенных алгоритмов в машинном обучении.

** показать значение производной в точках со слайдером **

** Полный график производной **

** Показать что производная слишком большая и нельзя так прыгать используя только ее ** 

** Показать что чем больше ошибка, тем больше производная, для MSE **

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib import cm

import matplotlib.pyplot as plt
import matplotlib as mpl
from ipywidgets import interact, IntSlider,  FloatSlider



def der_J(X, y, k):
    return np.mean((k*X - y)*X)*2

def plot_all_J_with_der(X, y):
    k_slider = FloatSlider(min=0, max=2, step=0.1, value=0.1)

    @interact(k=k_slider)
    def plot_all_J_with_der_inner(k):
        plt.figure(figsize=(8, 5))
        plt.title("Значение ошибки")
        plt.ylabel("Значение функции потерь")
        plt.xlabel("Значение коэффициента $k$")
        ks = np.linspace(-1, 2.65, 100)
        plt.plot(ks, [J(tmp_k, X, y) for tmp_k in ks], color='black')
        plt.scatter(k, J(k, X, y), s=50, marker='+', label="Текущее значение")
        
        k_0 = (k - J(k, X, y) / der_J(X, y, k))
        k_next = 5 if k > k_0 else -5
        
        plt.plot([ k_0, k_next], [0, J(k, X, y) + der_J(X, y, k)*(k_next - k)], label="Текущее значение")
        plt.legend()
        plt.grid()
        plt.xlim([-1, 3])
        plt.ylim([0, 0.15])
        plt.show() 

plot_all_J_with_der(X, y)

## Градиентный спуск

** переписать менее математично **

** объяснить альфу **

Как вызнаете, значение производной в точке равно значению тангенса угла наклона касательной в данной точке или же показывает растет ли функция или убывает.

Использую эту информацию мы можем понять где находится минимум и изменить значение $k$ в сторону минимума. 

* Если производная положительная (как касательная в точке 190000), то нам нужно уменьшать значение $k$. 
* Если производная отрицательная (как касательная в точке 180000), то нам нужно увеличить значение $k$.  

Таким образом сам алгоритм градиентного спуска можно описать следующим образом.

* Выбираем случайное значение для $k$
* Повторить пока не сойдется:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $k_{new} = k - \alpha \cdot \frac{d}{dk} J(k)$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $k = k_{new}$

Где $\alpha$ это коэффициент, который мы выбреем. 

In [ ]:
def gradient_function(k, X, y):
    N = X.shape[0]       
    grad = 2*np.sum((k * X - y) * X) / N                       
    return grad   

k = 170000
print(gradient_function(k, X, y))

In [ ]:
def gradient_descent(k_init, X, y, alpha, iters):
    k = k_init
    for i in range (0, iters):
        k = k - (alpha * gradient_function(k, X, y))
    return k

k_init = 170000 
alpha = 0.0001
iters = 10
gradient_descent(k_init, X, y, alpha, iters)

Аналитически подсчитаный k = 185072.4

Рассмотрим как ведет себя алгоритм с различными $\alpha$

** гифка или слйдер с лучшей визулизацией **

** добавить функию ошики и одновременно результурующий график. **

In [ ]:
Traice(X, y)

# Немного усложним пример

В качестве гипотезы мы использовали функцияю вида $f(X) = kX$.

Многие из вас могли заметить, что это не совсем линейная функция. Линейная фунция на самом деле выглядит так:

$f(X) = kX + b$.

Давайте изменим гипотезу и теперь будем использовать "настоящую" линейную функцию.

Но давайте немного изменим обозначения.

In [ ]:
import regression_helper

X, y = get_data()

In [ ]:
def f(X, k, b):
    return k*X + b

print(f(X=X, k=10, b=-10))

Перепишем функцию ошибки:

$\hat{y} = f(X) = kX + b$. То есть, $\hat{y}$ является предсказанными нами значениями для $X$. 
А настоящие значения будут равны $y$. 

Тогда ошибка на одном примере равна $j(k, b)$ на i-ом примере будет равна $j(k, b) = (\hat{y_i} - y_i)^2 = ((kX_i + b) - y_i)^2$

$J(k, b) = \frac{1}{N}\sum_{i=0}^{N}{(\hat{y_i} - y_i)^2}= \frac{1}{N} \sum_{i=0}^{N}{((kX_i + b) - y_i)^2} $

То есть теперь у нас ошибка зависит не только от $k$, но и от $b$.

In [ ]:
def linearn_loss_function(k, b, X, y):
    
    N = X.shape[0]       # получаем размер вектора столбца
    # или N = len(X)
   
    J = np.sum(((k*X + b) - y)**2)/N
    # или J = np.mean((k*X - y)**2)/N
    return J

k = 0
print(linearn_loss_function(k=k, b=10, X=X, y=y) )

In [ ]:
plot_linear_loss_in_3d(X, y)

## Градиент

Давайте введем понятие градиента.

Градиент фунцкии $\phi(x_0, x_1, \ldots x_N)$ от N переменных это 

N значений: $\frac{\delta \phi(x_0, x_1, \ldots x_N)}{\delta x_0}, \frac{\delta \phi(x_0, x_1, \ldots x_N)}{\delta x_1}, \ldots \frac{\delta \phi(x_0, x_1, \ldots x_N)}{\delta x_N}$

По сути - это координаты вектора, со значениями производных по разным осям. 


$\phi(x, y) = z$ 

$grad = \frac{\delta \phi(x, y)}{\delta x} \cdot \overleftarrow{i} + \frac{\delta \phi(x, y)}{\delta y} \cdot \overleftarrow{j}$


<img src="img/grad.png" alt="Drawing" style="width: 700px;"/>

** добавить рисунок вида сверху и прямо показать стрелки с градинтами для точки**

**Пример градиента от функции двух переменых**

Тогда определим нашу функцию ошибки от параметров $k, b$:

$J(k, b) = \frac{1}{N}\sum_{i=1}^{N}{(\hat{y_i} - y_i)^2}= \frac{1}{N} \sum_{i=1}^{N}{( (k X + b)  - y_i)^2}$

Для нахождения этих коэффициентов также используем градиентный спуск. Но теперь нам необходимо найти производную от функции ошибки для каждого параметра.

$\frac{\delta  J(k, b)}{\delta b} = \frac{2}{N}\sum_{i=1}^{N} ((k X_i + b)  - y_i)$ 

$\frac{\delta J(k, b)}{\delta k} = \frac{2}{N}\sum_{i=1}^{N} ((k X_i + b)  - y_i)X_i$ 


Тогда алгоритм градиентного спуска можно описать следующим образом:

* Выбираем случайное значение для $k$ и $b$
* Повторить пока не сойдется:

    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $b_{new} = b  - \alpha \cdot \frac{\delta }{\delta b} J(k, b)$ 
    
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $k_{new} = k - \alpha \cdot \frac{\delta }{\delta k} J(k, b)$ 
    
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $b = b_{new}$, 
    
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  $k = k_{new}$
    

Где $\alpha$ это коэффициент, который мы выбираем. 

In [ ]:



def plot_data(X, y, k, b):
    #create_base_plot()
    plt.scatter(X, y,  color='black', marker="o", s=50)   
    
    plt.plot([0, 0.5], [b, k*0.5 + b])
    #plt.ylim([0, 15])
    plt.show()



X, y =  get_data()
print(X)
print(y)
k = 0.2
b = 0.4
plot_data(X, y, k, b)

In [ ]:
def plot_linear_loss_in_3d(X, y):    
    angles1 = IntSlider(min=0, max=90, step=1, value=90, description='Вертикальное')
    angles2 = IntSlider(min=0, max=180, step=1, value=0, description='Горизонтальное')

    @interact(angle1=angles1, angle2=angles2)
    def plot_loss(angle1, angle2):
        fig = plt.figure(figsize=(15, 10))
        ax = fig.gca(projection='3d')

        # Make data.2.28
        ks = np.linspace(-4, 8, 50)#(-30, 30, 20)
        bs = np.linspace(-50, 50, 50)#(-100, 100, 20)#(-5, 3, 20)
        ks, bs = np.meshgrid(ks, bs)

        Z = np.zeros_like(ks)
        for i in range(len(ks)):
            for j in range(len(bs)):
                Z[i, j] = linearn_loss_function(ks[i, j], bs[i, j], X, y)

        ax.set_xlabel('Значение параметра $k$')
        ax.set_ylabel('Значение параметра $b$')
        ax.set_zlabel('Функция ошибки')
        
        surf = ax.plot_surface(ks, bs, Z,   linewidth=1, antialiased=False, cmap=cm.viridis_r)
        #surf = ax.plot_wireframe(ks, bs, Z,   cmap=cm.coolwarm)
        ax.view_init(angle1, angle2)
        
        #ax.set_xlim(-30, 30)
        #ax.set_ylim(0, 20)
        
        plt.show()

plot_linear_loss_in_3d(X, y)


In [ ]:
def gradient_function(k, b, X, y):
    N = X.shape[0]
    tmp = ( (k * X + b) - y ) *X
    #print(tmp)
    grad_b = 2*np.sum( ((k * X + b) - y)) / N                       
    grad_k = 2*np.sum( ((k * X + b) - y) * X) / N                       
    return np.array([grad_k, grad_b])

k = 5
b = 1
print(gradient_function(k, b, X, y))

In [ ]:

def gradient_descent(k_init, b_init, X, y, alpha, iters):
    k = k_init
    b = b_init
    for i in range (0, iters):
        gradients = gradient_function(k, b, X, y)
        k = k - (alpha * gradients[0])
        b = b - (alpha * gradients[1])
        print(i, k, b, gradients, linearn_loss_function(k, b, X, y) )
        #plot_data(X, y, k, b)
    
    return k, b





k_init = -5
b_init = 0
alpha = 0.75
iters = 250


k, b = gradient_descent(k_init, b_init, X, y, alpha, iters)
#print(k, b)
plot_data(X, y, k, b)

In [ ]:
b

In [ ]:
def lin_grad_linear(X, y, alpha, iters=20, k_init=0, b_init=0):    
    angles1 = IntSlider(min=0, max=90, step=1, value=90, description='Вертикальное')
    angles2 = IntSlider(min=0, max=180, step=1, value=0, description='Горизонтальное')
    
    k = None
    b = None
    
    ks = np.linspace(-5, 5, 20)
    bs = np.linspace(-5, 5, 20)
    ks, bs = np.meshgrid(ks, bs)
    Z = np.zeros_like(ks)
    for i in range(len(ks)):
        for j in range(len(ks)):
            Z[i, j] = linearn_loss_function(ks[i, j], bs[i, j], X, y)
    
    k = k_init
    b = b_init
    
    all_k = [k]
    all_b = [b]
    Js = [linearn_loss_function(k, b, X, y)]
    
    for i in range(iters):
        y_h = k*X + b - y 
        k = k - alpha * 2*sum(y_h * X) / len(X)
        b = b - alpha * 2*sum(y_h) / len(X)
        all_k.append(k)
        all_b.append(b)
        Js.append(linearn_loss_function(k, b, X, y))
    
    @interact(angle1=angles1, angle2=angles2)    
    def plot_trace(angle1, angle2):
        fig = plt.figure(figsize=(15, 10))
        ax = fig.gca(projection='3d')

        ax.set_xlabel('Значение коэффициента $k$')
        ax.set_ylabel('Значение коэффициента $b$')
        ax.set_zlabel('Функция ошибка')
        surf = ax.plot_wireframe(ks, bs, Z,  cmap=cm.coolwarm)

        ax.scatter(all_k[0], all_b[0], Js[0], c="yellow")
        ax.scatter(all_k[1:-1], all_b[1:-1], Js[1:-1], c="blue")
        ax.scatter(all_k[-1], all_b[-1], Js[-1], c="Red")
        ax.view_init(angle1, angle2)
        plt.show()
        
    return k, b

a=0.25
lin_grad_linear(X, y, alpha=a, iters=500, k_init=-4, b_init=5)